In [13]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [14]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [15]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [16]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [17]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [18]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [19]:
df['loc']

1     (9001 Collins Ave s 309, Surfside, FL 33154, U...
2     (100 South Pointe Dr, Miami Beach, FL 33139, U...
3     (10203 Collins Ave Unit 2103, Bal Harbour, FL ...
4     (8701 Collins Ave #601, Miami Beach, FL 33154,...
5     (791 Crandon Blvd, Miami, FL 33149, USA, (25.6...
6     (2015 Fisher Island Dr #2015, Miami Beach, FL ...
7     (789 Crandon Blvd APT 1003, Key Biscayne, FL 3...
8     (248 Washington Ave #3, Miami Beach, FL 33139,...
9     (18975 Collins Ave #603, Sunny Isles Beach, FL...
10    (15701 Collins Ave #3004, North Miami Beach, F...
Name: loc, dtype: object

In [20]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [21]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Four Seasons Surf Club\r9001 Collins Ave S-309...,6/9/2021,3/3/2022,198,"$18,500,000.00",4439,"$4,167.61",Carolina Lara-Arashiro,Pamela Diaz-De-Leon,Four Seasons Surf Club 9001 Collins Ave S-309 ...,Four Seasons Surf Club,9001 Collins Ave S-309 Surfside,"(9001 Collins Ave s 309, Surfside, FL 33154, U...","(25.8768949, -80.12146349999999, 0.0)",25.876895,-80.121463,0.0
2,Continuum South\r100 S Pointe Dr 1201/02\rMiam...,2/7/2022,2/28/2022,7,"$10,000,000.00",2792,"$3,581.66",Jorge Martinez,Jeri Jenkins,Continuum South 100 S Pointe Dr 1201/02 Miami ...,Continuum South,100 S Pointe Dr 1201/02 Miami Beach,"(100 South Pointe Dr, Miami Beach, FL 33139, U...","(25.766765, -80.13283659999999, 0.0)",25.766765,-80.132837,0.0
3,Oceana Bal Harbour\r10203 Collins Ave 2103\rBa...,7/19/2021,3/1/2022,198,"$7,900,000.00",4397,"$2,190.19",Senada Adzem,Oscar Arellano Jr.,Oceana Bal Harbour 10203 Collins Ave 2103 Bal ...,Oceana Bal Harbour,10203 Collins Ave 2103 Bal Harbour,"(10203 Collins Ave Unit 2103, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0
4,Eighty Seven Park\r8701 Collins Ave 601\rMiami...,9/16/2021,3/4/2022,148,"$6,200,000.00",2233,"$2,776.53",Michael Martirena,Jackson Keddell,Eighty Seven Park 8701 Collins Ave 601 Miami B...,Eighty Seven Park,8701 Collins Ave 601 Miami Beach,"(8701 Collins Ave #601, Miami Beach, FL 33154,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0
5,Ocean Tower Two Condo\r791 Crandon Blvd PH3\rK...,10/29/2021,3/2/2022,18,"$4,500,000.00",2718,"$1,655.63",Darin Spencer Raymond PA,Monica Slodarz,Ocean Tower Two Condo 791 Crandon Blvd PH3 Key...,Ocean Tower Two Condo,791 Crandon Blvd PH3 Key Biscayne,"(791 Crandon Blvd, Miami, FL 33149, USA, (25.6...","(25.6883371, -80.15793239999999, 0.0)",25.688337,-80.157932,0.0
6,Bayside Village East\r2015 Fisher Island Dr 20...,3/2/2022,3/2/2022,1,"$3,500,000.00",1912,"$1,830.54",Karla Abaunza,Karla Abaunza,Bayside Village East 2015 Fisher Island Dr 201...,Bayside Village East,2015 Fisher Island Dr 2015 Miami Beach,"(2015 Fisher Island Dr #2015, Miami Beach, FL ...","(25.7582172, -80.14165059999999, 0.0)",25.758217,-80.141651,0.0
7,Club Tower One Condo\r789 Crandon Blvd 1003\rK...,10/19/2021,2/28/2022,78,"$3,345,000.00",2500,"$1,338.00",Darin Spencer Raymond PA,Marta Iglesias,Club Tower One Condo 789 Crandon Blvd 1003 Key...,Club Tower One Condo,789 Crandon Blvd 1003 Key Biscayne,"(789 Crandon Blvd APT 1003, Key Biscayne, FL 3...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0
8,El Cyclon Condo\r248 Washington Ave 3\rMiami B...,1/11/2021,2/28/2022,381,"$3,212,500.00",3130,"$1,026.36",Jeffrey Miller,Nathan Zeder,El Cyclon Condo 248 Washington Ave 3 Miami Beach,El Cyclon Condo,248 Washington Ave 3 Miami Beach,"(248 Washington Ave #3, Miami Beach, FL 33139,...","(25.7721308, -80.13515989999999, 0.0)",25.772131,-80.135160,0.0
9,Residences by Armani Casa\r18975 Collins Ave 6...,8/23/2021,3/2/2022,153,"$3,200,000.00",2271,"$1,409.07",Paulina Sneider,Carolina Visbal LLC,Residences by Armani Casa 18975 Collins Ave 60...,Residences by Armani Casa,18975 Collins Ave 603 Sunny Isles Beach,"(18975 Collins Ave #603, Sunny Isles Beach, FL...","(25.9519943, -80.11997, 0.0)",25.951994,-80.119970,0.0
10,Ritz-Carlton\r15701 Collins Ave 3004\rSunny Is...,10/18/2021,3/1/2022,121,"$2,970,000.00",1735,"$1,711.82",Kevin Waissmann,Gabriela Dajer,Ritz-Carlton 15701 Collins Ave 3004 Sunny Isle...,Ritz-Carlton,15701 Collins Ave 3004 Sunny Isles Beach Beach,"(15701 Collins Ave #3004, North Miami Beach, F...","(25.9220146, -80.1217034, 0.0)",25.922015,-80.121703,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [22]:
#df.at[1,'Buyer Agent']=('Dora Puig, Carl Gambino')

In [23]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [24]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('February 27th - March 5th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [25]:
m.save('index.html')